# The time to beat is 60-80 seconds per file.

In [1]:
import sys, os
sys.path.insert(0,'/global/homes/b/bpb/metatlas/' )
sys.path.insert(1,'/global/project/projectdirs/metatlas/anaconda/lib/python2.7/site-packages' )
import pandas as pd
from metatlas import metatlas_objects as metob
from metatlas.helpers import dill2plots as dp
import numpy as np
import time
import copy
import dill
pd.options.display.max_colwidth = 100
pd.set_option('display.max_rows', 500)

('Metatlas live in ', '/global/homes/b/bpb/metatlas/metatlas')
you're running on fb2ee5119e85 at 172.17.0.2 


In [20]:
dp = reload(dp)
groups = dp.select_groups_for_analysis(name = '%nyogilipidsA%',
                                       most_recent = True,
                                       remove_empty = True,
                                       include_list = [], exclude_list = ['QC','Blank','extractioncontrol'])

49
0 20160504_KBL_NyogiLipidsA_NEG_72hr_BD_controlBD_noglc 2016-07-18 19:26:03
1 20160504_KBL_NyogiLipidsA_POS_24hr_BD_controlBD_noglc 2016-07-18 19:26:28
2 20160504_KBL_NyogiLipidsA_POS_48hr_BD_treatmentBD_glc 2016-07-18 19:26:04
3 20160504_KBL_NyogiLipidsA_POS_24hr_BD_treatmentBD_glc 2016-07-18 19:26:05
4 20160504_KBL_NyogiLipidsA_FPS_standards 2016-07-18 19:26:35
5 20160504_KBL_NyogiLipidsA_POS_OleicAcid_D9__ 2016-07-18 19:26:05
6 20160504_KBL_NyogiLipidsA_POS_PC_D31__ 2016-07-18 19:26:05
7 20160504_KBL_NyogiLipidsA_POS_48hr_BD_controlBD_noglc 2016-07-18 19:26:06
8 20160504_KBL_NyogiLipidsA_POS_48hr_BG_controlBG_glcglc 2016-07-18 19:26:07
9 20160504_KBL_NyogiLipidsA_NEG_12hr_BG_controlBG_glcglc 2016-07-18 19:26:08
10 20160504_KBL_NyogiLipidsA_NEG_84hr_BD_controlBD_noglc 2016-07-18 19:26:08
11 20160504_KBL_NyogiLipidsA_POS_PA_D31__ 2016-07-18 19:26:30
12 20160504_KBL_NyogiLipidsA_POS_84hr_BD_controlBD_noglc 2016-07-18 19:26:29
13 20160504_KBL_NyogiLipidsA_NEG_72hr_BD_treatmentBD_glc 

# Functions

In [21]:
def dataframe_container_from_metatlas_hdf5_file(my_file):
    data_df = pd.DataFrame()
    pd_h5_file  = pd.HDFStore(my_file)
    keys = pd_h5_file.keys()
    pd_h5_file.close()
    df_container = {}
    for k in keys:
        if ('ms' in k) and not ('_mz' in k):
            new_df = pd.read_hdf(my_file,k)
            df_container[k[1:]] = new_df
    return df_container


In [22]:
import os
df = pd.DataFrame()
for group in groups:
    for my_file in group.items:
        container = dataframe_container_from_metatlas_hdf5_file(my_file.hdf5_file)
        for k in container.keys():
            if ('ms2' in k) and ('pos' in k):
                container[k]['polarity'] = 'positive'
                container[k]['hdf5_file'] = os.path.basename(my_file.hdf5_file)
                df = pd.concat([df,container[k]],axis=0)
            elif ('ms2' in k) and ('neg' in k):
                container[k]['polarity'] = 'negative'
                container[k]['hdf5_file'] = os.path.basename(my_file.hdf5_file)
                df = pd.concat([df,container[k]],axis=0)


In [23]:
df.head()

,mz,i,rt,precursor_MZ,precursor_intensity,collision_energy,polarity,hdf5_file
0,54.924725,1707.031982,0.016607,174.960007,2790502.75,20.0,negative,20160504_C18_lipid_NEG_MSMS_KBL_Qex_MR_21_72hr_BD_a_controlBD_noglc___Run44.h5
1,55.410946,1535.511230,0.016607,174.960007,2790502.75,20.0,negative,20160504_C18_lipid_NEG_MSMS_KBL_Qex_MR_21_72hr_BD_a_controlBD_noglc___Run44.h5
2,57.473019,1776.081421,0.016607,174.960007,2790502.75,20.0,negative,20160504_C18_lipid_NEG_MSMS_KBL_Qex_MR_21_72hr_BD_a_controlBD_noglc___Run44.h5
3,71.284447,1834.279419,0.016607,174.960007,2790502.75,20.0,negative,20160504_C18_lipid_NEG_MSMS_KBL_Qex_MR_21_72hr_BD_a_controlBD_noglc___Run44.h5
4,76.574112,1567.962524,0.016607,174.960007,2790502.75,20.0,negative,20160504_C18_lipid_NEG_MSMS_KBL_Qex_MR_21_72hr_BD_a_controlBD_noglc___Run44.h5


In [24]:
grouped = df.groupby(['precursor_MZ','rt','polarity','precursor_intensity','collision_energy','hdf5_file']).aggregate(lambda x: tuple(x))
grouped.mz = grouped.mz.apply(list)
grouped.i = grouped.i.apply(list)
grouped = grouped.reset_index()
grouped['spectrum'] = map(lambda x,y:(x,y),grouped['mz'],grouped['i'])
grouped['spectrum'] = grouped['spectrum'].apply(lambda x: zip(x[0],x[1]))
grouped.drop(['mz','i'], axis=1, inplace=True)


In [7]:
grouped.head()

,precursor_MZ,rt,precursor_intensity,collision_energy,hdf5_file,spectrum
0,80.049644,1.029684,5.853951e+05,20.0,20160504_C18_lipid_POS_MSMS_KBL_Qex_MR_19_48hr_BD_c_treatmentBD_glc___Run82.h5,"[(53.0031, 147646.0), (53.0392, 7690.6), (53.9981, 6578.45), (54.0066, 1333.97), (63.5309, 1273...."
1,80.049644,7.208591,1.073722e+06,20.0,20160504_C18_lipid_POS_MSMS_KBL_Qex_MR_11_24hr_BD_c_treatmentBD_glc___Run85.h5,"[(50.0159, 1531.24), (51.0238, 10235.2), (53.0031, 316264.0), (53.0394, 58215.1), (53.9983, 1712..."
2,80.049652,4.948564,1.116939e+06,20.0,20160504_C18_lipid_POS_MSMS_KBL_Qex_MR_55_48hr_BG_c_controlBG_glcglc___Run91.h5,"[(51.0237, 3196.43), (53.0031, 295493.0), (53.0395, 34660.0), (53.9983, 15183.4), (54.0064, 6547..."
3,80.049652,7.676896,1.179940e+06,20.0,20160504_C18_lipid_POS_MSMS_KBL_Qex_MR_39_12hr_BG_c_controlBG_glcglc___Run97.h5,"[(51.0238, 13222.7), (52.0316, 1647.83), (52.3341, 1239.14), (53.003, 322221.0), (53.0394, 64183..."
4,80.049660,0.765142,5.261536e+05,20.0,20160504_C18_lipid_POS_MSMS_KBL_Qex_MR_3_0hr_BD_c_controlBD_noglc___Run88.h5,"[(53.0031, 137306.0), (53.0393, 8933.92), (53.9981, 7395.19), (55.2099, 1419.99), (56.07, 1265.7..."


In [9]:
target = 236.1485
min_intensity = 1e4
ppm = 10
def search_spectrum(spectrum,target,min_intensity,ppm):
    for s in spectrum:
        if (abs(s[0] - target) < target*ppm/1e6) & (s[1] > min_intensity):
            return True
    return False
grouped['has_target_fragment'] = grouped['spectrum'].apply(search_spectrum,args = (target,min_intensity,ppm))



In [10]:
idx = (grouped.has_target_fragment) & (grouped.rt > 3.5) & (grouped.rt < 7.5) & (grouped.precursor_MZ > 500)
print len(idx)
df_sqdg = grouped[idx]

321174


In [11]:
df_sqdg = df_sqdg.sort_values('precursor_intensity',ascending=False)

In [12]:
df_sqdg.head(10)

,precursor_MZ,rt,precursor_intensity,collision_energy,hdf5_file,spectrum,has_target_fragment
282295,734.591248,5.929809,640475008.0,20.0,20160504_C18_lipid_POS_MSMS_KBL_Qex_MR_4_0hr_BD_d_controlBD_noglc___Run145.h5,"[(53.2257, 318590.0), (56.1482, 300651.0), (58.066, 3.79567e+06), (59.0737, 425114.0), (60.0813,...",True
282421,736.606445,6.369270,638040576.0,20.0,20160504_C18_lipid_POS_MSMS_KBL_Qex_MR_31_84hr0hr_BDBG_b_treatmentBD_glc___Run100.h5,"[(51.4784, 415327.0), (54.2529, 400215.0), (58.0659, 2.8896e+06), (59.0737, 580927.0), (67.0127,...",True
282275,734.590759,5.923874,600906240.0,20.0,20160504_C18_lipid_POS_MSMS_KBL_Qex_MR_1_0hr_BD_a_controlBD_noglc___Run28.h5,"[(51.7392, 260099.0), (57.0341, 269280.0), (58.066, 5.04692e+06), (59.0739, 616133.0), (60.0817,...",True
282224,734.590271,5.937315,588514240.0,20.0,20160504_C18_lipid_POS_MSMS_KBL_Qex_MR_6_24hr_BD_b_controlBD_noglc___Run181.h5,"[(53.1054, 284154.0), (58.066, 3.11622e+06), (59.0739, 317586.0), (60.0816, 635078.0), (62.0885,...",True
282289,734.591003,5.921923,552017472.0,20.0,20160504_C18_lipid_POS_MSMS_KBL_Qex_MR_7_24hr_BD_c_controlBD_noglc___Run112.h5,"[(53.2219, 238582.0), (58.066, 2.19239e+06), (59.0739, 437669.0), (63.1671, 246552.0), (66.2774,...",True
285939,760.859985,5.980438,539685568.0,20.0,20160504_C18_lipid_POS_MSMS_KBL_Qex_MR_47_24hr_BG_c_controlBG_glcglc___Run94.h5,"[(121.082, 426690.0), (130.085, 604669.0), (144.091, 573229.0), (144.101, 1.14624e+07), (151.944...",True
282434,736.606567,6.362216,525579680.0,20.0,20160504_C18_lipid_POS_MSMS_KBL_Qex_MR_30_84hr0hr_BDBG_a_treatmentBD_glc___Run184.h5,"[(58.0659, 1.89263e+06), (59.074, 374397.0), (60.0816, 748919.0), (84.9048, 335022.0), (85.3387,...",True
285883,760.605652,6.011604,522524480.0,20.0,20160504_C18_lipid_POS_MSMS_KBL_Qex_MR_7_24hr_BD_c_controlBD_noglc___Run112.h5,"[(53.7186, 328855.0), (58.066, 3.41337e+06), (58.643, 382559.0), (60.0818, 510233.0), (60.3059, ...",True
282298,734.591797,5.920092,520063264.0,20.0,20160504_C18_lipid_POS_MSMS_KBL_Qex_MR_16_48hr_BD_d_controlBD_noglc___Run199.h5,"[(52.5973, 335096.0), (58.066, 3.03861e+06), (60.0817, 732028.0), (63.3717, 335876.0), (67.4622,...",True
282268,734.590698,5.941261,519522688.0,20.0,20160504_C18_lipid_POS_MSMS_KBL_Qex_MR_8_24hr_BD_d_controlBD_noglc___Run169.h5,"[(58.0659, 2.17665e+06), (58.1868, 291851.0), (59.53, 276671.0), (60.0816, 734307.0), (60.3298, ...",True


In [13]:
df_unique_sqdg = pd.DataFrame()
df_unique_sqdg = df_sqdg.iloc[[0]]
for i in df_sqdg.index:
    idx = abs(df_sqdg.loc[i,'precursor_MZ'] - df_unique_sqdg.precursor_MZ) < 0.02
    if sum(idx) == 0:
        #The mz is new so add it
        df_unique_sqdg = pd.concat([df_unique_sqdg,df_sqdg.loc[[i]]],axis = 0)
    else:
        #the mz has been seen before; so see if the rt is new
        if sum(abs(df_unique_sqdg[idx].rt - df_sqdg.loc[i,'rt']) < 0.1)==0:
            df_unique_sqdg = pd.concat([df_unique_sqdg,df_sqdg.loc[[i]]],axis = 0)

In [14]:
df_unique_sqdg.shape

(253, 7)

In [15]:
df_unique_sqdg.head(10)

,precursor_MZ,rt,precursor_intensity,collision_energy,hdf5_file,spectrum,has_target_fragment
282295,734.591248,5.929809,640475008.0,20.0,20160504_C18_lipid_POS_MSMS_KBL_Qex_MR_4_0hr_BD_d_controlBD_noglc___Run145.h5,"[(53.2257, 318590.0), (56.1482, 300651.0), (58.066, 3.79567e+06), (59.0737, 425114.0), (60.0813,...",True
282421,736.606445,6.369270,638040576.0,20.0,20160504_C18_lipid_POS_MSMS_KBL_Qex_MR_31_84hr0hr_BDBG_b_treatmentBD_glc___Run100.h5,"[(51.4784, 415327.0), (54.2529, 400215.0), (58.0659, 2.8896e+06), (59.0737, 580927.0), (67.0127,...",True
285939,760.859985,5.980438,539685568.0,20.0,20160504_C18_lipid_POS_MSMS_KBL_Qex_MR_47_24hr_BG_c_controlBG_glcglc___Run94.h5,"[(121.082, 426690.0), (130.085, 604669.0), (144.091, 573229.0), (144.101, 1.14624e+07), (151.944...",True
285883,760.605652,6.011604,522524480.0,20.0,20160504_C18_lipid_POS_MSMS_KBL_Qex_MR_7_24hr_BD_c_controlBD_noglc___Run112.h5,"[(53.7186, 328855.0), (58.066, 3.41337e+06), (58.643, 382559.0), (60.0818, 510233.0), (60.3059, ...",True
282305,736.340027,6.389369,501888992.0,20.0,20160504_C18_lipid_POS_MSMS_KBL_Qex_MR_55_48hr_BG_c_controlBG_glcglc___Run91.h5,"[(115.813, 267544.0), (130.085, 296396.0), (139.129, 289958.0), (144.102, 5.86519e+06), (146.005...",True
282158,734.589844,6.076521,358020448.0,20.0,20160504_C18_lipid_POS_MSMS_KBL_Qex_MR_49_48hr_BG_c_treatmentBG_glcnoglc___Run67.h5,"[(58.066, 1.83011e+06), (58.8998, 185754.0), (60.0814, 394615.0), (60.3834, 213134.0), (67.5189,...",True
285663,760.369995,5.983567,348493184.0,20.0,20160504_C18_lipid_POS_MSMS_KBL_Qex_MR_48_24hr_BG_d_controlBG_glcglc___Run175.h5,"[(117.631, 209881.0), (144.102, 6.04292e+06), (152.213, 445135.0), (162.112, 2.29687e+06), (184....",True
282493,736.849976,6.395312,309504608.0,20.0,20160504_C18_lipid_POS_MSMS_KBL_Qex_MR_52_48hr_BG_b_treatmentBG_glcnoglc___Run139.h5,"[(113.059, 277556.0), (113.876, 309288.0), (144.101, 3.38138e+06), (146.006, 488074.0), (149.076...",True
285410,758.589539,5.674184,270865568.0,20.0,20160504_C18_lipid_POS_MSMS_KBL_Qex_MR_28_84hr_BD_b_controlBD_noglc___Run187.h5,"[(56.4435, 198692.0), (57.1589, 200592.0), (58.066, 1.97225e+06), (59.0737, 360716.0), (60.0816,...",True
282031,732.574402,5.599775,270506080.0,20.0,20160504_C18_lipid_POS_MSMS_KBL_Qex_MR_46_24hr_BG_b_controlBG_glcglc___Run190.h5,"[(58.066, 2.17387e+06), (60.0815, 666431.0), (68.2846, 142680.0), (76.0465, 148414.0), (89.521, ...",True


In [16]:
df_unique_sqdg.to_csv('/global/homes/b/bpb/Downloads/dgts_potential_hits.csv')

In [17]:
from matplotlib import pyplot as plt
%matplotlib inline

In [18]:
print(plt.style.available)

[u'seaborn-darkgrid', u'seaborn-notebook', u'classic', u'seaborn-ticks', u'grayscale', u'bmh', u'seaborn-talk', u'dark_background', u'ggplot', u'fivethirtyeight', u'seaborn-colorblind', u'seaborn-deep', u'seaborn-whitegrid', u'seaborn-bright', u'seaborn-poster', u'seaborn-muted', u'seaborn-paper', u'seaborn-white', u'seaborn-pastel', u'seaborn-dark', u'seaborn-dark-palette']


In [19]:
for i in df_unique_sqdg.index:
    s = df_unique_sqdg.loc[i,'spectrum']
    mz = zip(*s)[0]
    intensity = zip(*s)[1]
    fig = plt.figure(figsize=(11,8.5))
    plt.style.use('seaborn-ticks')
    ax = plt.gca()
    ax.vlines(mz,intensity*0,intensity)
    ax.set_xlim(0,df_unique_sqdg.loc[i,'precursor_MZ']+5)
    ax.set_xlabel('m/z')
    ax.set_ylabel('intensity')
    sx = np.argsort(intensity)[::-1]
    labels = [1.001e9]
    for i_mz in sx:
        if np.min(np.abs(mz[i_mz] - labels)) > 0.1 and intensity[i_mz] > 0.02 * np.max(intensity):
            ax.annotate('%5.4f'%mz[i_mz], xy=(mz[i_mz], 1.01*intensity[i_mz]),rotation = 90, horizontalalignment = 'center', verticalalignment = 'left')
            labels.append(mz[i_mz])
    plt.tight_layout()
    L = plt.ylim()
    plt.ylim(L[0],L[1]*1.12)
    fig.savefig('/global/homes/b/bpb/Downloads/dgts_msms/msms_spectra_%d.png'%i)
    plt.close(fig)
